# Capstone Project

## Introduction
### In this capstone project I will be creating a program to compare the neighbourhoods of the two cities namely Brooklyn from New York and Downtown Toronto from Toronto and determine how similar or dissimilar they are. 
### I will be comparing all the venues , hotspots to visit, number of residential friendly places, etc. As we know  both Brooklyn and downtown Toronto are we very huge and diverse containing people from all around the world. These places have a lot of restaurants, cafes, pubs and so on. So by comparing these two neighbourhoods we can find a lot of similarities and see how these places vary. 
### I will use foursquare api to collect all the details and explore these regions and will be using k means to cluster different cateogaries of data and display it on the exact location using folium maps.
### Later I will be using the two maps to differentiate between the neighbourhoods and see how one varies from the other. This analysis is mainly done when people are moving from one place to another for various purposes. They would want to have the same facilities in the new neighbourhood as they have in their current neighbourhood.
### Thus by comparing various major cities based on its amenities and venues we make a proper analysis and visualize the data obtained using Data Science. 


## Data section
#### As I said in the introduction I will be creating a program to analyse Brooklyn and Downtown Toronto.I will be needing the data on Canadas neighbourhoodsLink had been obtained from previous labs from coursera:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
#### I will also be needing the New York dataset so that I can extract all the information from Brooklyn. I obtained This data set from the lab as well:https://cocl.us/new_york_dataset
#### I will also require a geospatial coordinate dataset which contains all the latitude and longitude information of the cities. This link was also available in the labs.http://cocl.us/Geospatial_data


In [1]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('All Required Libraries imported.')


All Required Libraries imported.


### preparing the canada dataframe

In [2]:
#web scraping
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_wiki=pd.read_html(path)
df_wiki



[    Postal code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [3]:
#making it into data frame
type(df_wiki)
neighborhood=df_wiki[0]
neighborhood.head()
neighborhood.rename(columns={'Postal code':'Postcode', 'Borough': 'Borough', 'Neighborhood': 'Neighborhood'}, inplace=True)
neighborhood.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
#removing not assigned values and cleaning

neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
neighborhood=pd.DataFrame(neighborhood)
neighborhood.set_index(['Postcode','Borough'],inplace=True)
merge_result = neighborhood.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
serial_wise=merge_result.reset_index()
serial_wise.loc[4, 'Neighborhood']='Queen\'s Park'
serial_wise.to_excel('wikipedia_table.xls')
df=pd.DataFrame(serial_wise)
df.head()


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park


In [5]:
#Retrieving longitude and lattitude from the geospatial cordinates csv "http://cocl.us/Geospatial_data"
df1=pd.read_csv("Geospatial_Coordinates.csv")
df1.rename(columns={'Postal Code':'Postcode'},inplace=True)
#merging the longitufes and lattitudes to previous dataframe
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
merge_columns=pd.merge(df, df1, left_on='Postcode', right_on='Postcode')
# Saving a copy of csv
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


### Lets select one of the highly active borough like downtown toronto and make a dataframe

In [6]:
#this data set will be used for comparision later
downtown_toronto_data = merge_columns[merge_columns['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data=downtown_toronto_data.drop(['Postcode'], axis=1)
downtown_toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Preparing the new york dataset

In [7]:
#the new york data set json file is already avilable from previous labs "https://cocl.us/new_york_dataset"
#converting json to dataframe
import json 
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [9]:
#addin the data into the frame
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
#making sure the data file has all data
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Like in toronto lets take a highly active city in new york like Brooklyn

In [11]:
# preparing a dataframe for Brooklyn
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


### Setting up the Foursquare API 

In [12]:
# instalizing all credentials and version of the api needed
CLIENT_ID = 'P5QITDQFTCI43G5YQSLFRBR5E25MMSEYQRRBQYRYIR2QEW3W'
CLIENT_SECRET = 'IYABW5A2QAIFGBEI1Q5MF3FLATHJKBDE1DLVFY1R0AQMHN02'
VERSION = '20180604'
limit = 20
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

CLIENT_ID:P5QITDQFTCI43G5YQSLFRBR5E25MMSEYQRRBQYRYIR2QEW3W
CLIENT_SECRET:IYABW5A2QAIFGBEI1Q5MF3FLATHJKBDE1DLVFY1R0AQMHN02


### Getting cordinates for of Downtown Toronto and Brooklyn.

In [16]:
#downto toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

c:\users\dp571\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [17]:
#Brooklyn
address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

c:\users\dp571\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


## Using Folium to visualize and compare the 2 locations

## Downtown Toronto

In [18]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [19]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

## Brooklyn

In [20]:
# let's visualizat Manhattan the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn)  
    
map_Brooklyn

In [21]:
# create map of Manhattan using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_Brooklyn)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_Brooklyn

## Statistical anaysis


### Exploring the venues in Downtown Toronto

In [24]:
#Make a function to collect all data
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # define the api request url and get request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # get the info
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
# usinfg the above function on each neighborhood we can create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Regent Park / Harbourfront
Queen's Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Rosedale
Stn A PO Boxes
St. James Town / Cabbagetown
First Canadian Place / Underground city
Church and Wellesley


In [26]:
#resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(359, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [27]:
#looking at all the venues in the neighbourhood
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,19,19,19,19,19,19
Church and Wellesley,20,20,20,20,20,20
Commerce Court / Victoria Hotel,20,20,20,20,20,20
First Canadian Place / Underground city,20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
Harbourfront East / Union Station / Toronto Islands,20,20,20,20,20,20


In [29]:
# finding all the diffrent cateogaries
print('There are {} diffrent categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 123 diffrent categories.


## Analyzing Each Neighborhood
### one hot encoding the data for downtown toronto

In [30]:
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]
downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
#grouping neighbourhood by the cateogaries
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [32]:
# getting the top 3 venues of each neighbourhood
num_top_venues = 3

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Farmers Market   0.1
1    Cocktail Bar   0.1
2        Beer Bar   0.1


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.30
1  Japanese Restaurant  0.10
2                  Spa  0.05


----Christie----
           venue  freq
0  Grocery Store  0.21
1           Café  0.16
2           Park  0.11


----Church and Wellesley----
                venue  freq
0  Mexican Restaurant  0.05
1      Breakfast Spot  0.05
2           Bookstore  0.05


----Commerce Court / Victoria Hotel----
         venue  freq
0         Café  0.25
1  Coffee Shop  0.10
2   Restaurant  0.10


----First Canadian Place / Underground city----
         venue  freq
0         Café  0.20
1  Coffee Shop  0.15
2   Restaurant  0.

In [33]:
# converting the data into a data frame
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Farmers Market,Beer Bar,French Restaurant,Seafood Restaurant,Park,Coffee Shop,Liquor Store,Bistro,Concert Hall
1,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Boutique,Bar,Harbor / Marina,Rental Car Location,Coffee Shop
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Modern European Restaurant,Sushi Restaurant,Bar,Sandwich Place,Seafood Restaurant,Ramen Restaurant,Spa,Italian Restaurant
3,Christie,Grocery Store,Café,Park,Coffee Shop,Gas Station,Candy Store,Nightclub,Athletics & Sports,Diner,Baby Store
4,Church and Wellesley,Bubble Tea Shop,Dance Studio,Burger Joint,Breakfast Spot,Park,Juice Bar,Bookstore,Pizza Place,Pub,Ramen Restaurant
5,Commerce Court / Victoria Hotel,Café,Restaurant,Coffee Shop,Gastropub,Museum,Gym,Pub,Beer Bar,Bakery,Gym / Fitness Center
6,First Canadian Place / Underground city,Café,Coffee Shop,Restaurant,Bakery,Gastropub,Deli / Bodega,Gluten-free Restaurant,Pub,American Restaurant,Gym
7,"Garden District, Ryerson",Café,Coffee Shop,Art Gallery,Movie Theater,Music Venue,Burrito Place,Burger Joint,Clothing Store,Plaza,Ramen Restaurant
8,Harbourfront East / Union Station / Toronto Is...,Café,Plaza,Park,Deli / Bodega,Salad Place,Italian Restaurant,Hotel,Skating Rink,Lake,Sporting Goods Shop
9,Kensington Market / Chinatown / Grange Park,Café,Vietnamese Restaurant,Mexican Restaurant,Bakery,Fish Market,Farmers Market,Dessert Shop,Coffee Shop,Cocktail Bar,Cheese Shop


## Clustering Neighborhoods

In [35]:
#clustering the data
kclusters = 5
downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 3, 0, 4, 1, 4, 4, 4, 1, 1])

In [36]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
downtown_toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Breakfast Spot,Historic Site,Distribution Center,Pub,Spa,Dessert Shop,Restaurant,Farmers Market
1,Downtown Toronto,Queen's Park,43.662301,-79.389494,3,Coffee Shop,Yoga Studio,Beer Bar,Distribution Center,Discount Store,Hobby Shop,Diner,Italian Restaurant,Creperie,Juice Bar
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Coffee Shop,Art Gallery,Movie Theater,Music Venue,Burrito Place,Burger Joint,Clothing Store,Plaza,Ramen Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Gastropub,Restaurant,Coffee Shop,Japanese Restaurant,Food Truck,BBQ Joint,Café,Middle Eastern Restaurant,Cosmetics Shop,Creperie
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Cocktail Bar,Farmers Market,Beer Bar,French Restaurant,Seafood Restaurant,Park,Coffee Shop,Liquor Store,Bistro,Concert Hall


In [37]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzing the Clusters


### Cluster 1 (Airport Lounge, Coffee Shop, Cafe, Restaurants & Grocery Store)

In [38]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",Café,Coffee Shop,Art Gallery,Movie Theater,Music Venue,Burrito Place,Burger Joint,Clothing Store,Plaza,Ramen Restaurant
10,Commerce Court / Victoria Hotel,Café,Restaurant,Coffee Shop,Gastropub,Museum,Gym,Pub,Beer Bar,Bakery,Gym / Fitness Center
11,University of Toronto / Harbord,Bookstore,Restaurant,Japanese Restaurant,Bakery,Yoga Studio,Dessert Shop,Café,College Gym,Comfort Food Restaurant,Beer Bar


### Cluster 2 (Gastropubs)

In [39]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park / Harbourfront,Coffee Shop,Bakery,Breakfast Spot,Historic Site,Distribution Center,Pub,Spa,Dessert Shop,Restaurant,Farmers Market
4,Berczy Park,Cocktail Bar,Farmers Market,Beer Bar,French Restaurant,Seafood Restaurant,Park,Coffee Shop,Liquor Store,Bistro,Concert Hall
8,Harbourfront East / Union Station / Toronto Is...,Café,Plaza,Park,Deli / Bodega,Salad Place,Italian Restaurant,Hotel,Skating Rink,Lake,Sporting Goods Shop
9,Toronto Dominion Centre / Design Exchange,Coffee Shop,Café,Restaurant,Gym / Fitness Center,Hotel,Gastropub,Deli / Bodega,Tea Room,Japanese Restaurant,American Restaurant
12,Kensington Market / Chinatown / Grange Park,Café,Vietnamese Restaurant,Mexican Restaurant,Bakery,Fish Market,Farmers Market,Dessert Shop,Coffee Shop,Cocktail Bar,Cheese Shop
16,St. James Town / Cabbagetown,Café,Restaurant,Bakery,Gastropub,Gift Shop,Diner,Deli / Bodega,Indian Restaurant,Italian Restaurant,Japanese Restaurant
18,Church and Wellesley,Bubble Tea Shop,Dance Studio,Burger Joint,Breakfast Spot,Park,Juice Bar,Bookstore,Pizza Place,Pub,Ramen Restaurant


### Cluster 3 (Cafes)

In [40]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Boutique,Bar,Harbor / Marina,Rental Car Location,Coffee Shop


### Cluster 4 (Coffee Shop, Cafe, Park & Japanese Restaurant)

In [41]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Queen's Park,Coffee Shop,Yoga Studio,Beer Bar,Distribution Center,Discount Store,Hobby Shop,Diner,Italian Restaurant,Creperie,Juice Bar


### Cluster 5 (Seafood, steakhouse, Hotel & Cafe)

In [42]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,St. James Town,Gastropub,Restaurant,Coffee Shop,Japanese Restaurant,Food Truck,BBQ Joint,Café,Middle Eastern Restaurant,Cosmetics Shop,Creperie
5,Central Bay Street,Coffee Shop,Japanese Restaurant,Modern European Restaurant,Sushi Restaurant,Bar,Sandwich Place,Seafood Restaurant,Ramen Restaurant,Spa,Italian Restaurant
6,Christie,Grocery Store,Café,Park,Coffee Shop,Gas Station,Candy Store,Nightclub,Athletics & Sports,Diner,Baby Store
7,Richmond / Adelaide / King,Asian Restaurant,Hotel,Concert Hall,Sushi Restaurant,Seafood Restaurant,Gym / Fitness Center,Speakeasy,Plaza,Steakhouse,Bar
14,Rosedale,Park,Trail,Playground,Creperie,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar
15,Stn A PO Boxes,Café,Cocktail Bar,Farmers Market,French Restaurant,Seafood Restaurant,Park,Jazz Club,Beer Bar,Restaurant,Concert Hall
17,First Canadian Place / Underground city,Café,Coffee Shop,Restaurant,Bakery,Gastropub,Deli / Bodega,Gluten-free Restaurant,Pub,American Restaurant,Gym


## Exploring Neighborhoods in Brooklyn.

In [43]:
#Make a function to collect all data
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # define the api request url and get request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # get the info
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
#using the above function on each neighborhood,we create a new dataframe called Brooklyn_venues
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude'],
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [45]:
#looking at all the venues in the neighbourhood
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,20,20,20,20,20,20
Bay Ridge,20,20,20,20,20,20
Bedford Stuyvesant,20,20,20,20,20,20
Bensonhurst,20,20,20,20,20,20
Bergen Beach,6,6,6,6,6,6
Boerum Hill,20,20,20,20,20,20
Borough Park,19,19,19,19,19,19
Brighton Beach,20,20,20,20,20,20
Broadway Junction,18,18,18,18,18,18


In [46]:
# finding all the diffrent cateogaries
print('There are {} uniques categories.'.format(len(Brooklyn_venues['Venue Category'].unique())))

There are 224 uniques categories.


## Analyzing the Neighborhoods
### one hot encoding the Brooklyn data

In [48]:
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
Brooklyn_onehot['Neighborhood'] =Brooklyn_venues['Neighborhood'] 
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]
Brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Entertainment Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,

In [49]:
#grouping neighbourhood by the cateogaries
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

In [50]:
# getting the top 3 venues of each neighbourhood
num_top_venues = 3

for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0  Italian Restaurant  0.10
1     Bubble Tea Shop  0.10
2    Sushi Restaurant  0.05


----Bay Ridge----
              venue  freq
0  Greek Restaurant   0.1
1               Spa   0.1
2     Grocery Store   0.1


----Bedford Stuyvesant----
         venue  freq
0         Café   0.1
1  Coffee Shop   0.1
2          Bar   0.1


----Bensonhurst----
              venue  freq
0    Ice Cream Shop   0.1
1  Sushi Restaurant   0.1
2       Pizza Place   0.1


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1      Baseball Field  0.17
2  Athletics & Sports  0.17


----Boerum Hill----
                    venue  freq
0                     Bar   0.1
1  Furniture / Home Store   0.1
2             Coffee Shop   0.1


----Borough Park----
         venue  freq
0         Bank  0.21
1  Pizza Place  0.16
2     Pharmacy  0.11


----Brighton Beach----
                      venue  freq
0              Dessert Shop  0.05
1      Other Great Ou

In [51]:
# converting the data into a data frame
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Italian Restaurant,Bubble Tea Shop,Ice Cream Shop,Bakery,Dessert Shop,Park,Pizza Place,Dim Sum Restaurant,Coffee Shop,Rental Car Location
1,Bay Ridge,Spa,Grocery Store,Greek Restaurant,Breakfast Spot,Bagel Shop,Chinese Restaurant,Lounge,Bookstore,Middle Eastern Restaurant,Coffee Shop
2,Bedford Stuyvesant,Café,Coffee Shop,Bar,Fried Chicken Joint,BBQ Joint,New American Restaurant,Boutique,Cocktail Bar,Gift Shop,Juice Bar
3,Bensonhurst,Ice Cream Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Hotpot Restaurant,Liquor Store,Chinese Restaurant,Noodle House,Flower Shop,Bakery
4,Bergen Beach,Harbor / Marina,Playground,Baseball Field,Donut Shop,Athletics & Sports,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop
5,Boerum Hill,Coffee Shop,Furniture / Home Store,Bar,Bakery,Spa,Martial Arts Dojo,Sushi Restaurant,Cocktail Bar,Kids Store,Concert Hall
6,Borough Park,Bank,Pizza Place,Pharmacy,Deli / Bodega,Coffee Shop,American Restaurant,Hotel,Eastern European Restaurant,Fast Food Restaurant,Café
7,Brighton Beach,Pharmacy,Other Great Outdoors,Taco Place,Beach,Russian Restaurant,Korean Restaurant,Eastern European Restaurant,Restaurant,Non-Profit,Supermarket
8,Broadway Junction,Diner,Donut Shop,Sandwich Place,Ice Cream Shop,Gas Station,Discount Store,Recording Studio,Nightclub,Caribbean Restaurant,Business Service
9,Brooklyn Heights,Yoga Studio,Pet Store,Gym / Fitness Center,Playground,Pizza Place,Mexican Restaurant,Japanese Restaurant,History Museum,Gym,Thai Restaurant


### CLUSTERING NEIGHBORHOODS


## Brooklyn

In [55]:
#clustering the data
kclusters = 5
Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 1])

In [56]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
Brooklyn_merged = Brooklyn_data
Brooklyn_merged['Cluster Labels'] = kmeans.labels_
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,1,Spa,Grocery Store,Greek Restaurant,Breakfast Spot,Bagel Shop,Chinese Restaurant,Lounge,Bookstore,Middle Eastern Restaurant,Coffee Shop
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Ice Cream Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Hotpot Restaurant,Liquor Store,Chinese Restaurant,Noodle House,Flower Shop,Bakery
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Pizza Place,Latin American Restaurant,Mexican Restaurant,Bank,Bakery,Grocery Store,Creperie,Café,Mobile Phone Shop,Fried Chicken Joint
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Café,Mexican Restaurant,Yoga Studio,Pizza Place,Polish Restaurant,Record Shop,Cocktail Bar,French Restaurant,Furniture / Home Store
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Pizza Place,Lounge,Italian Restaurant,Bakery,Gym,Fish Market,Men's Store,Furniture / Home Store,Martial Arts Dojo,Chinese Restaurant


In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## EXAMINE CLUSTERS

### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

## Brooklyn

### Residential

In [58]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Windsor Terrace,Diner,Café,Grocery Store,Wine Shop,Bagel Shop,Sushi Restaurant,Coffee Shop,Bookstore,American Restaurant,Middle Eastern Restaurant
52,Ocean Parkway,Playground,Donut Shop,Bus Station,Bus Stop,Sake Bar,Restaurant,Pizza Place,Paper / Office Supplies Store,Men's Store,Sushi Restaurant


### Commercial Places

In [59]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Grocery Store,Greek Restaurant,Breakfast Spot,Bagel Shop,Chinese Restaurant,Lounge,Bookstore,Middle Eastern Restaurant,Coffee Shop
1,Bensonhurst,Ice Cream Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Hotpot Restaurant,Liquor Store,Chinese Restaurant,Noodle House,Flower Shop,Bakery
2,Sunset Park,Pizza Place,Latin American Restaurant,Mexican Restaurant,Bank,Bakery,Grocery Store,Creperie,Café,Mobile Phone Shop,Fried Chicken Joint
3,Greenpoint,Bar,Café,Mexican Restaurant,Yoga Studio,Pizza Place,Polish Restaurant,Record Shop,Cocktail Bar,French Restaurant,Furniture / Home Store
4,Gravesend,Pizza Place,Lounge,Italian Restaurant,Bakery,Gym,Fish Market,Men's Store,Furniture / Home Store,Martial Arts Dojo,Chinese Restaurant
5,Brighton Beach,Pharmacy,Other Great Outdoors,Taco Place,Beach,Russian Restaurant,Korean Restaurant,Eastern European Restaurant,Restaurant,Non-Profit,Supermarket
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Cosmetics Shop,Steakhouse,Organic Grocery,Coffee Shop,Burger Joint,Bank,Convenience Store
8,Flatbush,Caribbean Restaurant,Mexican Restaurant,Coffee Shop,Bank,Plaza,Bagel Shop,Sandwich Place,Lounge,Juice Bar,Donut Shop
9,Crown Heights,Pizza Place,Museum,Café,Fried Chicken Joint,Salon / Barbershop,Moving Target,Supermarket,Coffee Shop,Bookstore,Bakery
10,East Flatbush,Department Store,Park,Moving Target,Supermarket,Fast Food Restaurant,Caribbean Restaurant,Chinese Restaurant,Food & Drink Shop,Pharmacy,Wine Shop


### Tourist Areas & Hubs

In [60]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Mill Basin,Pizza Place,Bagel Shop,Japanese Restaurant,Bank,Pharmacy,Pet Store,Deli / Bodega,Peruvian Restaurant,Chinese Restaurant,Food Truck


### Center Acivity

In [61]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Prospect Park South,Caribbean Restaurant,Mexican Restaurant,Latin American Restaurant,Grocery Store,Food Truck,Bank,Pizza Place,Electronics Store,Clothing Store,Miscellaneous Shop


### Cultural & Going Out Places

In [62]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Pizza Place,Diner,Outlet Store,Restaurant,Russian Restaurant,Miscellaneous Shop,Buffet
17,Bedford Stuyvesant,Café,Coffee Shop,Bar,Fried Chicken Joint,BBQ Joint,New American Restaurant,Boutique,Cocktail Bar,Gift Shop,Juice Bar
24,Park Slope,Bagel Shop,Coffee Shop,Ice Cream Shop,Sporting Goods Shop,Accessories Store,Bookstore,Burger Joint,Dessert Shop,Frozen Yogurt Shop,Furniture / Home Store
25,Cypress Hills,Pizza Place,Fried Chicken Joint,Ice Cream Shop,Fast Food Restaurant,Latin American Restaurant,Discount Store,Bank,Supermarket,Gas Station,Baseball Field
27,Starrett City,Pizza Place,Bus Stop,Chinese Restaurant,Donut Shop,Supermarket,Cosmetics Shop,American Restaurant,Pharmacy,Caribbean Restaurant,Fish & Chips Shop
28,Canarsie,Chinese Restaurant,Caribbean Restaurant,Grocery Store,Gym,Shipping Store,Asian Restaurant,Farm,Factory,Falafel Restaurant,Wings Joint
29,Flatlands,Pharmacy,Caribbean Restaurant,Fast Food Restaurant,Fried Chicken Joint,Chinese Restaurant,Bar,Nightclub,Electronics Store,Lounge,Paper / Office Supplies Store
56,Rugby,Caribbean Restaurant,Bank,Grocery Store,Wings Joint,Deli / Bodega,Pharmacy,Diner,Chinese Restaurant,Seafood Restaurant,Mobile Phone Shop
58,New Lots,Pizza Place,Fried Chicken Joint,Plaza,Grocery Store,Discount Store,Park,Chinese Restaurant,Bus Stop,Spanish Restaurant,Breakfast Spot
59,Paerdegat Basin,Food,Moving Target,Harbor / Marina,Asian Restaurant,Gym,Wings Joint,Event Space,Flower Shop,Fish Market,Fish & Chips Shop


# RESULTS

### After analysing the data both the neighbourhoods have a high tourist attraction , lot of restaurants and cafes. They also have a lot of good residential areas like parks etc and proper amenities places like airports and train.
### The major difference is that they have different historical places and different origins. This includes monuments ,parliamentary buildings and so on.


# Observations & Discussion

### Brooklyn is bigger than downtown Toronto it has 3 times more venues than its counterpart as we can see on the map. Not only that the people are diverse as it is evident from the diverse array of restaurants available and the places to visit.
### The airport and other transport facilities are nearby and many residential areas are available in Brooklyn than in downtown Toronto.


# Conclusion

### Based on a the analysis the Brooklyn neighbourhood is more suitable for a highly active and busy person who always wants to enjoy and keep traveling without any financial barriers.
### On the other hand if you want a simpler life with balance of nature and entertainment downtown Toronto is the best option and is also a solution financially.
